In [1]:
import gc
import psutil
import logging
import time
import json
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from collections import defaultdict
import os
from mpl_toolkits.mplot3d import Axes3D
from collections import defaultdict, Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm  # Ensure Counter is imported

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import json

conversations_file_path = '/content/drive/MyDrive/HSN-619 Project/Dataset/conversations.json'
corpus_file_path = '/content/drive/MyDrive/HSN-619 Project/Dataset/corpus.json'
users_file_path = '/content/drive/MyDrive/HSN-619 Project/Dataset/users.json'
jsonl_file_1_path = '/content/drive/MyDrive/HSN-619 Project/Dataset/info.parsed.jsonl'
jsonl_file_2_path = '/content/drive/MyDrive/HSN-619 Project/Dataset/utterances.jsonl'

In [4]:
with open(users_file_path, 'r') as file:
    for line in file:
        try:
            data = json.loads(line)
            print(json.dumps(data, indent=2))
        except json.JSONDecodeError:
            continue

{
  "u1121": {
    "character_name": "ELAINE",
    "movie_idx": "m74",
    "movie_name": "ghostbusters ii",
    "gender": "?",
    "credit_pos": "?"
  },
  "u6336": {
    "character_name": "H.L.",
    "movie_idx": "m423",
    "movie_name": "lone star",
    "gender": "?",
    "credit_pos": "?"
  },
  "u8145": {
    "character_name": "CARCANOGUES",
    "movie_idx": "m553",
    "movie_name": "the man who wasn't there",
    "gender": "?",
    "credit_pos": "?"
  },
  "u7513": {
    "character_name": "RANDY",
    "movie_idx": "m507",
    "movie_name": "scream",
    "gender": "m",
    "credit_pos": "16"
  },
  "u8396": {
    "character_name": "JULIA",
    "movie_idx": "m569",
    "movie_name": "the thin man",
    "gender": "f",
    "credit_pos": "11"
  },
  "u8513": {
    "character_name": "PARIS",
    "movie_idx": "m577",
    "movie_name": "tomorrow never dies",
    "gender": "?",
    "credit_pos": "?"
  },
  "u6859": {
    "character_name": "GUY",
    "movie_idx": "m458",
    "movie_name":

In [5]:
with open(corpus_file_path, 'r') as file:
    for line in file:
        try:
            data = json.loads(line)
            print(json.dumps(data, indent=2))
        except json.JSONDecodeError:
            continue

{
  "movie_metadata": {
    "m0": {
      "title": "10 things i hate about you",
      "url": "http://www.dailyscript.com/scripts/10Things.html",
      "release_year": "1999",
      "rating": "6.90",
      "votes": "62847",
      "genre": "['comedy', 'romance']"
    },
    "m1": {
      "title": "1492: conquest of paradise",
      "url": "http://www.hundland.org/scripts/1492-ConquestOfParadise.txt",
      "release_year": "1992",
      "rating": "6.20",
      "votes": "10421",
      "genre": "['adventure', 'biography', 'drama', 'history']"
    },
    "m2": {
      "title": "15 minutes",
      "url": "http://www.dailyscript.com/scripts/15minutes.html",
      "release_year": "2001",
      "rating": "6.10",
      "votes": "25854",
      "genre": "['action', 'crime', 'drama', 'thriller']"
    },
    "m3": {
      "title": "2001: a space odyssey",
      "url": "http://www.scifiscripts.com/scripts/2001.txt",
      "release_year": "1968",
      "rating": "8.40",
      "votes": "163227",
      

In [6]:
with open(jsonl_file_1_path, 'r') as f:
    for i, line in enumerate(f):
        if i == 100:  # Stop after 10 lines
            break
        record = json.loads(line.strip())  # Parse JSON line
        print(record)

{'id': 'L1045', 'value': [{'rt': 1, 'toks': [{'tok': 'They', 'tag': 'PRP', 'dep': 'nsubj', 'up': 1, 'dn': []}, {'tok': 'do', 'tag': 'VBP', 'dep': 'ROOT', 'dn': [0, 2, 3]}, {'tok': 'not', 'tag': 'RB', 'dep': 'neg', 'up': 1, 'dn': []}, {'tok': '!', 'tag': '.', 'dep': 'punct', 'up': 1, 'dn': []}]}]}
{'id': 'L1044', 'value': [{'rt': 2, 'toks': [{'tok': 'They', 'tag': 'PRP', 'dep': 'nsubj', 'up': 1, 'dn': []}, {'tok': 'do', 'tag': 'VBP', 'dep': 'aux', 'up': 2, 'dn': [0]}, {'tok': 'to', 'tag': 'TO', 'dep': 'ROOT', 'dn': [1, 3]}, {'tok': '!', 'tag': '.', 'dep': 'punct', 'up': 2, 'dn': []}]}]}
{'id': 'L985', 'value': [{'rt': 1, 'toks': [{'tok': 'I', 'tag': 'PRP', 'dep': 'nsubj', 'up': 1, 'dn': []}, {'tok': 'hope', 'tag': 'VBP', 'dep': 'ROOT', 'dn': [0, 2, 3]}, {'tok': 'so', 'tag': 'RB', 'dep': 'advmod', 'up': 1, 'dn': []}, {'tok': '.', 'tag': '.', 'dep': 'punct', 'up': 1, 'dn': []}]}]}
{'id': 'L984', 'value': [{'rt': 1, 'toks': [{'tok': 'She', 'tag': 'PRP', 'dep': 'nsubj', 'up': 1, 'dn': []}, 

In [7]:
with open(jsonl_file_2_path, 'r') as f:
    for i, line in enumerate(f):
        if i == 100:  # Stop after 10 lines
            break
        record = json.loads(line.strip())  # Parse JSON line
        print(record)

{'id': 'L1045', 'root': 'L1044', 'text': 'They do not!', 'user': 'u0', 'meta': {'movie_id': 'm0'}, 'reply-to': 'L1044', 'timestamp': None}
{'id': 'L1044', 'root': 'L1044', 'text': 'They do to!', 'user': 'u2', 'meta': {'movie_id': 'm0'}, 'reply-to': None, 'timestamp': None}
{'id': 'L985', 'root': 'L984', 'text': 'I hope so.', 'user': 'u0', 'meta': {'movie_id': 'm0'}, 'reply-to': 'L984', 'timestamp': None}
{'id': 'L984', 'root': 'L984', 'text': 'She okay?', 'user': 'u2', 'meta': {'movie_id': 'm0'}, 'reply-to': None, 'timestamp': None}
{'id': 'L925', 'root': 'L924', 'text': "Let's go.", 'user': 'u0', 'meta': {'movie_id': 'm0'}, 'reply-to': 'L924', 'timestamp': None}
{'id': 'L924', 'root': 'L924', 'text': 'Wow', 'user': 'u2', 'meta': {'movie_id': 'm0'}, 'reply-to': None, 'timestamp': None}
{'id': 'L872', 'root': 'L870', 'text': "Okay -- you're gonna need to learn how to lie.", 'user': 'u0', 'meta': {'movie_id': 'm0'}, 'reply-to': 'L871', 'timestamp': None}
{'id': 'L871', 'root': 'L870', 't

In [8]:
with open(corpus_file_path, 'r') as f:
    movie_metadata = json.load(f)

with open(users_file_path, 'r') as f:
    character_data = json.load(f)

with open(jsonl_file_2_path, 'r') as f:
    conversation_data = [json.loads(line) for line in f]

with open(jsonl_file_1_path, 'r') as f:
    utterance_data = [json.loads(line) for line in f]

MULTI-EDGE NETWORK PLOT

In [9]:
multi_edge_network_plot_path = "/content/drive/MyDrive/HSN-619 Project/movie characters multi-edge network plot"

# Step 1: Map user_id to character_name
user_to_character = {user: data["character_name"] for user, data in character_data.items()}

# Step 2: Build conversation graph by movie
movie_conversations = defaultdict(list)

# Create a lookup table for conversation IDs
conversation_lookup = {conv["id"]: conv for conv in conversation_data}

# Map conversations to movies
for conv in conversation_data:
    movie_id = conv["meta"]["movie_id"]
    speaker = user_to_character.get(conv["user"], conv["user"])  # Use character name or fallback to user_id
    reply_to = conv.get("reply-to")
    if reply_to and reply_to in conversation_lookup:
        reply_speaker = user_to_character.get(conversation_lookup[reply_to]["user"], conversation_lookup[reply_to]["user"])
        movie_conversations[movie_id].append((speaker, reply_speaker))

In [ ]:
# Step 3: Create graphs for each movie and visualize in 3D
for movie_id, edges in movie_conversations.items():
    # Count edge frequencies
    edge_counts = Counter(edges)

    # Get movie title from movie_metadata directly
    movie_info = movie_metadata.get(movie_id, {})
    movie_title = movie_info.get("title", f"Movie ID: {movie_id}")  # Fallback to movie_id if not found

    # Create a graph
    G = nx.MultiGraph()
    for edge, weight in edge_counts.items():
        G.add_edge(edge[0], edge[1], weight=weight)  # Add edge with weight

    # 3D Layout for graph
    pos_3d = nx.spring_layout(G, dim=3, seed=42)  # Seed for consistent layout
    pos_3d_array = {node: (pos[0], pos[1], pos[2]) for node, pos in pos_3d.items()}

    # 3D Visualization
    fig = plt.figure(figsize=(16, 12))
    ax = fig.add_subplot(111, projection="3d")

    # Remove grid and axis
    ax.grid(False)
    ax.set_axis_off()

    # Draw nodes
    for node, (x, y, z) in pos_3d_array.items():
        ax.scatter(x, y, z, color="red", s=150)  # Node size and color
        # Move text labels slightly away from the nodes (by adding an offset)
        ax.text(x, y, z + 0.1, s=node, fontsize=9, fontweight="bold", ha="center", color="black", zorder=5)

    # Draw edges with thickness based on weight
    for edge in G.edges(data=True):
        x_vals = [pos_3d_array[edge[0]][0], pos_3d_array[edge[1]][0]]
        y_vals = [pos_3d_array[edge[0]][1], pos_3d_array[edge[1]][1]]
        z_vals = [pos_3d_array[edge[0]][2], pos_3d_array[edge[1]][2]]

        # Thickness based on edge weight (conversation frequency)
        edge_thickness = edge[2].get("weight", 1) * 0.12  # Scale for better visualization
        ax.plot(x_vals, y_vals, z_vals, color="gray", linewidth=edge_thickness)

    # Title and save the plot
    ax.set_title(f"Character Network for '{movie_title}'", fontsize=14, pad=20)

    # Save the plot to Google Drive
    sanitized_title = movie_title.replace(" ", "_").replace(":", "").replace("'", "")  # Clean the title for filenames
    file_path = f"{multi_edge_network_plot_path}/{sanitized_title}_network.png"
    plt.savefig(file_path, dpi=300, bbox_inches='tight')
    plt.close(fig)  # Close the plot to save memory

DIRECTED NETWORK PLOT

In [10]:
save_dir = '/content/drive/MyDrive/HSN-619 Project/movie characters directed network plot'  # Path inside your Google Drive
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Step 1: Initialize data structures to store word counts per character
character_word_counts = defaultdict(lambda: defaultdict(int))  # {movie_id: {character_name: word_count}}

# Step 2: Count words per character in each conversation
for conv in conversation_data:
    movie_id = conv["meta"]["movie_id"]
    speaker = user_to_character.get(conv["user"], conv["user"])  # Get character name
    reply_to = conv.get("reply-to")

    # Assuming conversation text (replace with actual conversation text if available)
    conversation_text = "sample conversation text"  # Replace with actual text data

    # Count words in the conversation (split by spaces, ignoring punctuation)
    word_count = len(conversation_text.split())

    # Add to the character's word count for the current movie
    character_word_counts[movie_id][speaker] += word_count

# Step 3: Create directed edges based on the word count comparison
directed_network = defaultdict(list)

for movie_id, char_counts in character_word_counts.items():
    # For each movie, compare word counts between characters
    for speaker, speaker_count in char_counts.items():
        for other_speaker, other_count in char_counts.items():
            if speaker != other_speaker:
                # If speaker A utters more words than speaker B, draw a directed edge from A to B
                if speaker_count > other_count:
                    directed_network[movie_id].append((speaker, other_speaker))

In [ ]:
# Step 4: Visualize the directed graph for each movie and save the plots
for movie_id, edges in directed_network.items():
    movie_info = movie_metadata.get(movie_id, {})
    movie_title = movie_info.get("title", f"Movie ID: {movie_id}")

    # Create directed graph
    G = nx.DiGraph()
    G.add_edges_from(edges)

    # 3D Layout for graph
    pos_3d = nx.spring_layout(G, dim=3, seed=42)
    pos_3d_array = {node: (pos[0], pos[1], pos[2]) for node, pos in pos_3d.items()}

    # 3D Visualization
    fig = plt.figure(figsize=(16, 12))
    ax = fig.add_subplot(111, projection="3d")

    # Remove grid and axis
    ax.grid(False)
    ax.set_axis_off()

    # Draw nodes
    for node, (x, y, z) in pos_3d_array.items():
        ax.scatter(x, y, z, color="red", s=150)
        ax.text(x, y, z + 0.1, s=node, fontsize=9, fontweight="bold", ha="center", color="black", zorder=5)

        # Draw directed edges with arrow tips
    for edge in G.edges():
        x_vals = [pos_3d_array[edge[0]][0], pos_3d_array[edge[1]][0]]
        y_vals = [pos_3d_array[edge[0]][1], pos_3d_array[edge[1]][1]]
        z_vals = [pos_3d_array[edge[0]][2], pos_3d_array[edge[1]][2]]

        # Plot the edge with a line
        ax.plot(x_vals, y_vals, z_vals, color="gray", linewidth=1.5)

        # Add arrow to show direction (from A to B)
        ax.quiver(x_vals[0], y_vals[0], z_vals[0], x_vals[1] - x_vals[0], y_vals[1] - y_vals[0], z_vals[1] - z_vals[0],
                  arrow_length_ratio=0.1, color="blue", linewidth=0.1)

    # Title and save plot
    plot_filename = f"{movie_title}_directed_network.png"
    plot_path = os.path.join(save_dir, plot_filename)
    ax.set_title(f"Directed Character Network for '{movie_title}'", fontsize=14, pad=20)

    # Save the plot to Google Drive
    plt.savefig(plot_path, bbox_inches='tight')
    plt.close()  # Close the plot to free memory

Selecting Random Graph Models: MULTI-EDGE MODEL

In [ ]:
# Ensure the output directory exists
os.makedirs(multi_edge_network_plot_path, exist_ok=True)

# Function to generate Erdos-Renyi random graph
def generate_erdos_renyi(graph):
    V = graph.number_of_nodes()
    E = graph.number_of_edges()
    er_graph = nx.MultiGraph()

    nodes = list(graph.nodes())
    for _ in range(E):
        u, v = np.random.choice(nodes, size=2, replace=False)
        er_graph.add_edge(u, v)

    return er_graph

# Function to generate Configuration Model random graph
def generate_configuration_model(graph):
    degrees = [degree for _, degree in graph.degree()]
    cm_graph = nx.configuration_model(degrees, create_using=nx.MultiGraph(), seed=42)
    cm_graph.remove_edges_from(nx.selfloop_edges(cm_graph))  # Remove self-loops
    return cm_graph

# Function to generate Preferential Attachment random graph
def generate_preferential_attachment(graph, p=0.5):
    V = graph.number_of_nodes()
    E = graph.number_of_edges()
    node_list = [node for node, degree in graph.degree() for _ in range(degree)]

    pa_graph = nx.MultiGraph()
    for _ in range(E):
        u = np.random.choice(node_list)
        v = np.random.choice(node_list)
        pa_graph.add_edge(u, v)

    return pa_graph

# Function to generate Chung-Lu random graph
def generate_chung_lu(graph):
    degrees = np.array([degree for _, degree in graph.degree()])
    degree_sum = degrees.sum()

    node_indices = {node: i for i, node in enumerate(graph.nodes())}  # Map node names to indices
    cl_graph = nx.MultiGraph()

    for u in graph.nodes():
        for v in graph.nodes():
            if u != v:  # No self-loops
                u_index, v_index = node_indices[u], node_indices[v]  # Get indices for u and v
                prob = (degrees[u_index] * degrees[v_index]) / degree_sum  # Use indices to access degrees
                if np.random.rand() < prob:
                    cl_graph.add_edge(u, v)

    return cl_graph

# Function to visualize graphs in 3D
def visualize_graph_3d(graph, title, file_path):
    # 3D Layout for graph
    pos_3d = nx.spring_layout(graph, dim=3, seed=42)
    pos_3d_array = {node: (pos[0], pos[1], pos[2]) for node, pos in pos_3d.items()}

    fig = plt.figure(figsize=(16, 12))
    ax = fig.add_subplot(111, projection="3d")
    ax.grid(False)
    ax.set_axis_off()

    # Draw nodes
    for node, (x, y, z) in pos_3d_array.items():
        ax.scatter(x, y, z, color="red", s=150)
        ax.text(x, y, z + 0.1, s=node, fontsize=9, fontweight="bold", ha="center", color="black", zorder=5)

    # Draw edges
    for edge in graph.edges(data=True):
        x_vals = [pos_3d_array[edge[0]][0], pos_3d_array[edge[1]][0]]
        y_vals = [pos_3d_array[edge[0]][1], pos_3d_array[edge[1]][1]]
        z_vals = [pos_3d_array[edge[0]][2], pos_3d_array[edge[1]][2]]

        edge_thickness = edge[2].get("weight", 1) * 0.12  # Default weight if not present
        ax.plot(x_vals, y_vals, z_vals, color="gray", linewidth=edge_thickness)

    ax.set_title(title, fontsize=14, pad=20)
    plt.savefig(file_path, dpi=300, bbox_inches='tight')
    plt.close(fig)

# Main process for each movie
for movie_id, edges in movie_conversations.items():
    # Count edge frequencies
    edge_counts = Counter(edges)

    # Get movie title and clean it for filenames
    movie_info = movie_metadata.get(movie_id, {})
    movie_title = movie_info.get("title", f"Movie ID: {movie_id}")
    sanitized_title = movie_title.replace(" ", "_").replace(":", "").replace("'", "")

    # Create the original graph
    G_original = nx.MultiGraph()
    for edge, weight in edge_counts.items():
        G_original.add_edge(edge[0], edge[1], weight=weight)

    # Generate random graphs
    random_graphs = {
        "Erdos-Renyi": generate_erdos_renyi(G_original),
        "Configuration Model": generate_configuration_model(G_original),
        "Preferential Attachment": generate_preferential_attachment(G_original),
        "Chung-Lu": generate_chung_lu(G_original),
    }

    # Save plots for each graph
    visualize_graph_3d(
        G_original,
        f"Original Network for '{movie_title}'",
        f"{multi_edge_network_plot_path}/{sanitized_title}_original.png"
    )
    for model_name, random_graph in random_graphs.items():
        visualize_graph_3d(
            random_graph,
            f"{model_name} Network for '{movie_title}'",
            f"{multi_edge_network_plot_path}/{sanitized_title}_{model_name.replace(' ', '_')}.png"
        )

Selecting Random Graph Models: DIRECTED NETWORK MODEL

In [ ]:
# Function for Erdos-Rényi Directed Graph
def erdos_renyi_directed(graph):
    V = graph.number_of_nodes()
    E = graph.number_of_edges()
    p = E / (V * (V - 1))  # Probability for directed edges
    nodes = list(graph.nodes())
    er_graph = nx.DiGraph()

    for u in nodes:
        for v in nodes:
            if u != v and np.random.random() < p:
                er_graph.add_edge(u, v)

    return er_graph

# Function for Chung-Lu Directed Graph
def chung_lu_directed(graph):
    indegrees = np.array([deg for _, deg in graph.in_degree()])
    outdegrees = np.array([deg for _, deg in graph.out_degree()])
    total_weight = np.sum(outdegrees[:, None] * indegrees[None, :])

    cl_graph = nx.DiGraph()
    nodes = list(graph.nodes())

    for i, u in enumerate(nodes):
        for j, v in enumerate(nodes):
            if u != v:
                pij = (outdegrees[i] * indegrees[j]) / total_weight
                if np.random.random() < pij:
                    cl_graph.add_edge(u, v)

    return cl_graph

# Function for Fast Reciprocal Directed (FRD) Graph
def frd_directed(graph):
    indegrees = np.array([deg for _, deg in graph.in_degree()])
    outdegrees = np.array([deg for _, deg in graph.out_degree()])
    reciprocated_edges = sum(1 for u, v in graph.edges() if graph.has_edge(v, u))
    total_edges = graph.number_of_edges()
    E_reciprocal = reciprocated_edges // 2

    frd_graph = nx.DiGraph()
    nodes = list(graph.nodes())

    # Add reciprocated edges
    for _ in range(E_reciprocal):
        i, j = random.choices(nodes, weights=indegrees * outdegrees, k=2)
        if i != j and not frd_graph.has_edge(i, j) and not frd_graph.has_edge(j, i):
            frd_graph.add_edge(i, j)
            frd_graph.add_edge(j, i)

    # Add single directed edges
    for _ in range(total_edges - 2 * E_reciprocal):
        i = random.choices(nodes, weights=outdegrees, k=1)[0]
        j = random.choices(nodes, weights=indegrees, k=1)[0]
        if i != j and not frd_graph.has_edge(i, j):
            frd_graph.add_edge(i, j)

    return frd_graph

# Function for Preferential Attachment Directed Graph
def preferential_attachment_directed(graph, p=0.2):
    V = graph.number_of_nodes()
    outdegrees = [deg for _, deg in graph.out_degree()]
    pa_graph = nx.DiGraph()
    nodes = list(graph.nodes())
    current_indegrees = {node: 0 for node in nodes}

    for i, outdegree in enumerate(outdegrees):
        for _ in range(outdegree):
            if random.random() < p:
                target = random.choice([n for n in nodes if n != nodes[i]])
            else:
                targets = list(current_indegrees.keys())
                target_weights = [current_indegrees[t] + 1 for t in targets]
                target = random.choices(targets, weights=target_weights, k=1)[0]

            pa_graph.add_edge(nodes[i], target)
            current_indegrees[target] += 1

    return pa_graph

# Function to visualize and save graphs
def visualize_and_save(graph, title, save_path):
    pos_3d = nx.spring_layout(graph, dim=3, seed=42)
    pos_3d_array = {node: (pos[0], pos[1], pos[2]) for node, pos in pos_3d.items()}

    fig = plt.figure(figsize=(16, 12))
    ax = fig.add_subplot(111, projection="3d")
    ax.grid(False)
    ax.set_axis_off()

    # Draw nodes
    for node, (x, y, z) in pos_3d_array.items():
        ax.scatter(x, y, z, color="red", s=150)
        ax.text(x, y, z + 0.1, s=node, fontsize=9, fontweight="bold", ha="center", color="black", zorder=5)

    # Draw edges
    for edge in graph.edges():
        x_vals = [pos_3d_array[edge[0]][0], pos_3d_array[edge[1]][0]]
        y_vals = [pos_3d_array[edge[0]][1], pos_3d_array[edge[1]][1]]
        z_vals = [pos_3d_array[edge[0]][2], pos_3d_array[edge[1]][2]]

        ax.plot(x_vals, y_vals, z_vals, color="gray", linewidth=1.5)
        ax.quiver(
            x_vals[0], y_vals[0], z_vals[0],
            x_vals[1] - x_vals[0], y_vals[1] - y_vals[0], z_vals[1] - z_vals[0],
            arrow_length_ratio=0.1, color="blue", linewidth=0.1
        )

    ax.set_title(title, fontsize=14, pad=20)
    plt.savefig(save_path, bbox_inches="tight")
    plt.close()

# Main loop to process each movie
for movie_id, edges in directed_network.items():
    movie_info = movie_metadata.get(movie_id, {})
    movie_title = movie_info.get("title", f"Movie ID: {movie_id}")
    sanitized_title = movie_title.replace(" ", "_").replace(":", "").replace("'", "")

    # Create original directed graph
    G_directed = nx.DiGraph()
    G_directed.add_edges_from(edges)

    # Generate directed random graphs
    directed_models = {
        "Erdos-Renyi": erdos_renyi_directed(G_directed),
        "Chung-Lu": chung_lu_directed(G_directed),
        "Fast Reciprocal Directed": frd_directed(G_directed),
        "Preferential Attachment": preferential_attachment_directed(G_directed),
    }

    # Save plots
    for model_name, model_graph in directed_models.items():
        plot_filename = f"{sanitized_title}_{model_name.replace(' ', '_')}_directed.png"
        plot_path = os.path.join(save_dir, plot_filename)
        visualize_and_save(model_graph, f"{model_name} Directed Network for '{movie_title}'", plot_path)


TRAINING GRAPH CLASSIFIER

In [12]:
# Helper: Feature Extraction
def extract_features(graph):
    """Extract feature vector for a graph."""
    features = {
        "num_nodes": graph.number_of_nodes(),
        "num_edges": graph.number_of_edges(),
        "average_degree": np.mean([d for _, d in graph.degree()]),
        "density": nx.density(graph),
        "clustering_coefficient": nx.average_clustering(graph),
    }
    return np.array(list(features.values()))

# Generate Graph Models for Multi-Edge Networks
def generate_erdos_renyi(graph):
    n = graph.number_of_nodes()
    m = graph.number_of_edges()
    p = 2 * m / (n * (n - 1))
    return nx.gnm_random_graph(n, m)

def generate_configuration_model(graph):
    degree_sequence = [d for _, d in graph.degree()]
    return nx.configuration_model(degree_sequence, create_using=nx.MultiGraph())

def generate_preferential_attachment(graph):
    m = int(graph.number_of_edges() / graph.number_of_nodes())
    return nx.barabasi_albert_graph(graph.number_of_nodes(), m)

def generate_chung_lu(graph):
    degree_sequence = [d for _, d in graph.degree()]
    return nx.expected_degree_graph(degree_sequence, selfloops=False)

# Generate Graph Models for Directed Networks
def erdos_renyi_directed(graph):
    V = graph.number_of_nodes()
    E = graph.number_of_edges()
    p = E / (V * (V - 1))
    edges = np.random.rand(V, V) < p
    np.fill_diagonal(edges, 0)
    er_graph = nx.from_numpy_matrix(edges, create_using=nx.DiGraph())
    return nx.relabel_nodes(er_graph, {i: n for i, n in enumerate(graph.nodes())})

def chung_lu_directed(graph):
    indegrees = np.array([deg for _, deg in graph.in_degree()])
    outdegrees = np.array([deg for _, deg in graph.out_degree()])
    total_weight = outdegrees.sum() * indegrees.sum()
    pij = np.outer(outdegrees, indegrees) / total_weight
    edges = np.random.rand(len(indegrees), len(indegrees)) < pij
    np.fill_diagonal(edges, 0)
    cl_graph = nx.from_numpy_matrix(edges, create_using=nx.DiGraph())
    return nx.relabel_nodes(cl_graph, {i: n for i, n in enumerate(graph.nodes())})

def frd_directed(graph):
    nodes = list(graph.nodes())
    indegrees = np.array([deg for _, deg in graph.in_degree()])
    outdegrees = np.array([deg for _, deg in graph.out_degree()])
    reciprocated_edges = sum(1 for u, v in graph.edges() if graph.has_edge(v, u)) // 2
    total_edges = graph.number_of_edges()

    frd_graph = nx.DiGraph()
    for _ in range(reciprocated_edges):
        i, j = np.random.choice(nodes, size=2, replace=False, p=outdegrees * indegrees)
        frd_graph.add_edges_from([(i, j), (j, i)])
    for _ in range(total_edges - 2 * reciprocated_edges):
        i = np.random.choice(nodes, p=outdegrees)
        j = np.random.choice(nodes, p=indegrees)
        if i != j and not frd_graph.has_edge(i, j):
            frd_graph.add_edge(i, j)
    return frd_graph

def preferential_attachment_directed(graph, p=0.2):
    nodes = list(graph.nodes())
    outdegrees = [deg for _, deg in graph.out_degree()]
    pa_graph = nx.DiGraph()
    current_indegrees = np.zeros(len(nodes), dtype=int)

    for i, outdegree in enumerate(outdegrees):
        for _ in range(outdegree):
            if np.random.random() < p:
                target = np.random.choice(nodes)
            else:
                target_weights = current_indegrees + 1
                target = np.random.choice(nodes, p=target_weights / target_weights.sum())
            if nodes[i] != target:
                pa_graph.add_edge(nodes[i], target)
                current_indegrees[nodes.index(target)] += 1
    return pa_graph

# Main Workflow
def train_and_evaluate_classifier(networks, metadata, model_type="multi-edge"):
    all_features = []
    all_labels = []
    model_generators = {
        "multi-edge": {
            "Erdos-Renyi": generate_erdos_renyi,
            "Configuration Model": generate_configuration_model,
            "Preferential Attachment": generate_preferential_attachment,
            "Chung-Lu": generate_chung_lu,
        },
        "directed": {
            "Erdos-Renyi": erdos_renyi_directed,
            "Chung-Lu": chung_lu_directed,
            "FRD": frd_directed,
            "Preferential Attachment": preferential_attachment_directed,
        },
    }

    # Process each movie
    for movie_id, edges in tqdm(networks.items(), desc="Processing movies"):
        movie_info = metadata.get(movie_id, {})
        G_original = (
            nx.MultiGraph(edges)
            if model_type == "multi-edge"
            else nx.DiGraph(edges)
        )
        original_features = extract_features(G_original)

        # Generate random graphs
        for model_name, generator in model_generators[model_type].items():
            for _ in range(100):  # Generate 100 samples
                random_graph = generator(G_original)
                features = extract_features(random_graph)
                all_features.append(features)
                all_labels.append(model_name)

        # Include original graph for testing later
        all_features.append(original_features)
        all_labels.append("Original")

    # Prepare data
    feature_array = np.array(all_features)
    label_array = np.array(all_labels)
    train_features, test_features, train_labels, test_labels = train_test_split(feature_array, label_array, test_size=0.2, random_state=42, stratify=label_array)

    # Train classifier
    classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    classifier.fit(train_features, train_labels)

    # Evaluate
    predictions = classifier.predict(test_features)
    accuracy = accuracy_score(test_labels, predictions)
    print(f"Classifier Test Accuracy: {accuracy:.4f}")

    return classifier

# Example Usage
# Replace `multi_edge_network` and `directed_network` with actual data
multi_edge_network = {}  # Replace with actual multi-edge network data
directed_network = {}  # Replace with actual directed network data
movie_metadata = {}  # Replace with actual metadata

# Train on multi-edge networks
print("Training on Multi-Edge Networks...")
multi_edge_classifier = train_and_evaluate_classifier(
    multi_edge_network, movie_metadata, model_type="multi-edge"
)

# Train on directed networks
print("Training on Directed Networks...")
directed_classifier = train_and_evaluate_classifier(
    directed_network, movie_metadata, model_type="directed"
)

Training on Multi-Edge Networks...


Processing movies: 0it [00:00, ?it/s]


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.